### 워크플로우 이니시

In [1]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
import os
import logging
from core import logging_utils, utils

log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR,
    "CRITICAL": logging.CRITICAL
}

log_level = log_levels.get(os.environ.get("LOG_LEVEL", "INFO").upper(), logging.INFO)

logging_utils.setup_logging(
    log_dir="logs",
    log_level=log_level,
    max_bytes=10 * 1024 * 1024, 
    backup_count=5, 
    console_output=True 
)

logging_utils.cleanup_old_logs(log_dir="logs", days_to_keep=7)


15:50:20 - core.logging_utils - INFO - Logging initialized - logs directory: c:\Users\kakao\Desktop\AI_Agent\Semantic Layer\logs
15:50:20 - core.logging_utils - INFO - Log rotation configured: max 10.0MB per file, keeping 5 backups


In [ ]:
resource_db = load_resource_specs("./ResourceSpec/TokenSpec.yaml")
validator = TokenValidator(resource_db)

# Chain Validator 인스턴스 생성
chain_check = SpecChainValidator(validator)

# 최초 토큰 생성
valid_token = Token(
    trace_id = "dfhou3898dfalss28fhs", # 향후 랜덤 생성기 필요
    content = {
        "text" : "Samsung Elec Risk Analysis",
        "risk_score" : 0.96
    }
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_A = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_001",
    description="금융 뉴스 감성 분석",
    type=TaskType.PYTHON_FUNC, 
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V1",
    output_spec_id="RS_RISK_TOKEN_V2"
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_B = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_002",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2",

)


In [ ]:

task_C = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_003",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2"

)

chain_check.validate_link(task_A, task_B)

15:55:36 - ChainValidator - INFO - [LINK] ID 일치 RS_RISK_TOKEN_V2 -> RS_RISK_TOKEN_V2


True

### 프로세스 테스트

In [3]:
# 최초 선언시 아이디 선언
process_test = Process("ojs_test_process")

# (ORANGE) 나중에 Process 선언문 읽고 아래 action이 자동 수행되는 로직이 필요
process_test.tasks
process_test.add_task(task_A)
process_test.add_task(task_B)
process_test.add_link(task_A, task_B)

# 그래프 구조 완성된거 확인 가능
process_test.graph
process_test.compile(chain_validator=chain_check)

15:30:25 - ChainValidator - INFO - [LINK] ID 일치 RS_RISK_TOKEN_V2 -> RS_RISK_TOKEN_V2
15:30:25 - Proc_ojs_test_process - INFO - [COMPILE SUCCESS] Process Topology 정상.


True

##### ANDOR Join Test

In [5]:
proc = Process("RISK_MODEL_FLOW")

# 2. 태스크 등록
proc.add_task(task_A) # 재무
proc.add_task(task_B) # 거시
proc.add_task(task_C) # 통합 모델

NameError: name 'task_C' is not defined

In [7]:
proc.add_link(task_A, task_C) # 재무 -> 통합
proc.add_link(task_B, task_C) # 거시 -> 통합

In [9]:
proc.predecessors

defaultdict(set, {'TASK_TEST_003': {'TASK_TEST_001', 'TASK_TEST_002'}})

### /engine/engine.py

In [ ]:
import importlib
import logging
from dataclasses import dataclass, field
from typing import Any, Optional, Dict, List
from datetime import datetime


In [ ]:

@dataclass
class FiringResult:
    task_id: str    
    success: bool
    message: str
    new_token: Optional[Any] = None
    elapsed_ms: float = 0.0
    routes_taken: int = 0


class ExecEngine:
    """
    핵심 프로세스 실행 엔진
    CSPN의 Transition(Task)을 실행하고, 토큰을 변환/생성하는 역할 수행
    
    내부 절차 : 가드 체크 -> 입력 확인 -> 함수 실행 -> 아웃풋 확인 -> 토큰 갱신 -> 라우팅
    """

